# CASO DE NEGOCIO: Modelamiento de Retención de Clientes en Scholastic Travel Company

## Sinopsis

Scholastic Travel Company (STC) quiere usar los datos de sus clientes para predecir quién comprará un paquete de viaje el próximo año. Los casos presentan datos de 2.389 clientes, donde el conjunto A contiene varios campos de perfil y el conjunto B contiene información adicional del Net Promoted Score (NPS)

Este notebook ha sido preparado con información provista en el Caso de Negocios de Harvard UV7579 (Agosto 23, 2018), Retention Modeling at Scholastic Travel Company, elaborado por Anton Ovchinnikov, profesor de Management Science, Operations Management y Customer Analytics, en la escuela de negocios Smith, de Queen’s University en Canadá.

## Objetivos

Los objetivos del ejercicio son:

1.   Entender el problema de negocio presentado y cómo este problema se convierte en un problema de analítica de datos
2.   Aplicar técnicas adecuadas en preparación de datos para clasificación
3.   Aplicar técnicas adecuadas en limpieza de datos: lidiar con valores faltantes, categorías extrañas
4.   Practicar con herramientas avanzadas de analítica de datos como regresión logística (incluyendo la selección de variables Recursive Feature Elimination para Python) y máquinas de vectores de soporte (SVM)
5.   Entender las métricas de casificación más comunes: matriz de confusión, curva ROC, Área bajo la curva AUC, para poder comparar modelos analíticos

Los principales pasos a seguir son:

1.   Asegurar que los paquetes y librerías necesarios están instalados (e instalarlos si no lo están)
2.   Cargar los paquetes y librerías necesarios
3.   Cargar los datos
4.   "Limpiar" los datos: necesitaremos (4.1) convertir algunas características en tipos correctos, (4.2) combinar categorías de variables, (4.3) arreglar los valores que faltan, y (4.4) crear dummies (one hot encoding) para las características no numéricas
5.   Definir el objetivo (la variable que intentamos predecir) y la matriz de características (todas las demás, excepto el ID)
6.   Dividir los datos en entrenamiento y prueba
7.   Entrenar (ajustar) el modelo con los datos de entrenamiento 
8.   Aplicarlo a los datos de prueba 
9.   Calcular las métricas del modelo y ajustar los hiperparámetros para mejorar esas métricas
10.  Exportar las predicciones para la toma de decisiones


Consideraremos varios modelos de analítica de datos:

1. MÁQUINAS DE VECTORES DE SOPORTE: un método intermedio entre las regresiones y los árboles que se ajusta a planos de menor dimensión para separar los datos en clases con el máximo margen entre ellas



### Paso 0: Para iniciar ... 

# Pasos 1 y 2: Instalar y cargar los paquetes y librerías necesarias

In [ ]:
# Paso 1: Chequear el ambiente Anaconda y los paquetes/librerías instalados
# import sys
# !conda env list
# !conda list
# !conda update --all

# Descargar e instalar pandas, numpy, scikit-learn. Podría ser necesario hacerlos desde el prompt de Anaconda
# !conda install pandas # pandas includes numpy 
# !conda install scikit-learn

# Paso 2: Cargar los paquetes y librerías necesarios 

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer

import warnings
warnings.filterwarnings("ignore")

# Paso 3: Cargar los datos

In [ ]:
# Paso 3: Cargar los datos desde el archivo CSV en el dataframe llamado df.

df = pd.read_csv(r"C:\Users\carlo\Downloads\04 CSV data -- STC(A)_numerical dates (2).csv", sep=";", header = 0)  
df.head() # show the "head" -- first 5 rows of the data; note, these are rows 0...4

# Paso 4: "Limpiar" los datos

En este caso, los datos se dejan a propósito ligeramente "sucios", es decir, se limpian previamente en cierta medida, pero para efectos de aprendizaje todavía quedan algunos elementos de datos "sucios":

- Algunos campos de datos (variables, características, columnas) tienen tipos incorrectos, por ejemplo, deberían convertirse de números a categorías.

- Algunas variables categóricas tienen demasiados valores (niveles), y algunos de los niveles son demasiado raros: por ejemplo, sólo hay un grupo de Bahamas -- estos datos deberían fusionarse en una categoría más poblada

- Faltan algunos datos y hay que sustituirlos o imputarlos

- Para concluir la limpieza de los datos, tendremos que crear, por supuesto, variables ficticias ("one hot encodig") para las variables categóricas 

In [ ]:
# Limpieza de datos -- parte 1: convertir tipos de datos incorrectos

# Algunos de los tipos de datos que maneja Python:
# int -- número entero (e.g., 5)
# float -- número fraccionario (e.g., 5.25)
# object, str -- text (string). Un texto que contiene varios valores no ordenados (e.g., M/F)

df.info() # Para chequear qué tipo de datos tenemos 

# Otros tipos de datos en el paquete pandas:
# category -- categoricos, igual que "factor" in R (e.g., red/green/blue, or M/F: una lista con varios valores no ordenados)
# datetime -- fecha y hora (e.g., 01.01.2020)
# bool -- binario (e.g.? yes/no, 1/0)

In [ ]:
# Limpieza de los datos -- parte 1: conversión de tipos de datos que deberían ser categóricos

df['From.Grade'] = df['From.Grade'].astype('category')
df['To.Grade'] = df['To.Grade'].astype('category')
df['Group.State'] = df['Group.State'].astype('category')
df['Is.Non.Annual.'] = df['Is.Non.Annual.'].astype('category')
df['Travel.Type'] = df['Travel.Type'].astype('category')
df['Poverty.Code'] = df['Poverty.Code'].astype('category')
df['CRM.Segment'] = df['CRM.Segment'].astype('category')
df['School.Type'] = df['School.Type'].astype('category')
df['Parent.Meeting.Flag'] = df['Parent.Meeting.Flag'].astype('category')
df['MDR.Low.Grade'] = df['MDR.Low.Grade'].astype('category')
df['MDR.High.Grade'] = df['MDR.High.Grade'].astype('category')
df['School.Sponsor'] = df['School.Sponsor'].astype('category')
df['SchoolGradeTypeLow'] = df['SchoolGradeTypeLow'].astype('category')
df['SchoolGradeTypeHigh'] = df['SchoolGradeTypeHigh'].astype('category')
df['GroupGradeTypeLow'] = df['GroupGradeTypeLow'].astype('category')
df['GroupGradeTypeHigh'] = df['GroupGradeTypeHigh'].astype('category')
df['MajorProgramCode'] = df['MajorProgramCode'].astype('category')
df['SingleGradeTripFlag'] = df['SingleGradeTripFlag'].astype('category')
df['SchoolSizeIndicator'] = df['SchoolSizeIndicator'].astype('category')
df['Retained.in.2012.'] = df['Retained.in.2012.'].astype('category')

df['Region'] = df['Region'].astype('category')
df['Special.Pay'] = df['Special.Pay'].astype('category')
df['Income.Level'] = df['Income.Level'].astype('category')
df['SPR.Product.Type'] = df['SPR.Product.Type'].astype('category')
df['SPR.New.Existing'] = df['SPR.New.Existing'].astype('category')

df.info() # Chequeemos los resultados

In [ ]:
# Limpieza de datos -- parte 2: combinar categorías poco frecuentes ("niveles")

df['Group.State'].value_counts() # Tomemos el ejemplo de la variable Group.State




In [ ]:


# Presentamos una función personalizada que llamaremos CombineRareCategories
# esta función tiene dos argumentos: el nombre del dataframe (data) y el número mínimo de puntos de datos para seguir 
# siendo una categoría separada (mincount)
# esta función recorrerá todas las columnas del marco de datos y combinará todas las categorías que aparezcan en los 
# datos menos que el número mínimo de veces en (Other)

def CombineRareCategories(data, mincount):
    for col in data.columns:
        if (type(data[col][0]) == str):
            for index, row in pd.DataFrame(data[col].value_counts()).iterrows():
                if ( row[0] < mincount):
                    df[col].replace(index, 'Other_' + col, inplace = True)
                else:
                    None

# Aplicamos esta función a variables con mincount=10                    
CombineRareCategories(df, 10)        

df[0:10] # Chequeamos los resultados

In [ ]:
# Limpieza de datos -- parte 3: Reemplazo/Imputación de datos faltantes

pd.DataFrame(df).isna().sum() # Chequeamos si hay datos faltantes

In [ ]:
pd.set_option('display.max_rows', 500)
# Hay datos no disponibles.  Revisemos el porcentaje de los mismos.
nan_percentage = pd.DataFrame(df).isna().sum() / len(pd.DataFrame(df))
missing_percentage_df = pd.DataFrame({'column_name': pd.DataFrame(df).columns, 'percent_missing': nan_percentage}).reset_index(drop=True)
missing_percentage_df

In [ ]:
len(pd.DataFrame(df))

In [ ]:
# Método:
# Variables Categóricas: agregar una nueva categoría 'missing_value' (como si fuera un nuevo color, o género)
# Variables Numéricas: reemplazar con la mediana (o la media, o el valor más frecuente, etc.) Un método alterno es
# ejecutar una imputación, see here: https://scikit-learn.org/stable/modules/impute.html 
# + agregamos columnas sustitutas indicando que el valor ha sido imputado

# creación de variables sustitutas
for col in df:
    if df[col].isna().sum() != 0: 
        df[col + '_surrogate'] = df[col].isna().astype(int)

# fijación de variables categóricas
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(df.select_dtypes(exclude=['int64','float64']))
df[df.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(df.select_dtypes(exclude=['int64','float64']))
           
# fijación de variables numéricas 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(df.select_dtypes(include=['int64','float64']))
df[df.select_dtypes(include=['int64','float64']).columns] = imputer.transform(df.select_dtypes(include=['int64','float64']))



In [ ]:
# Examinemos los resultados para la variable "Poverty.Code"
df[['Poverty.Code','Poverty.Code_surrogate']]


In [ ]:
# Examinemos los resultados para la variable "Special.Pay"
df[['Special.Pay','Special.Pay_surrogate']]
df[['FirstMeeting','FirstMeeting_surrogate']]


In [ ]:
df.shape
df.info()

In [ ]:
# Limpieza de datos -- parte 4: creación de dummies para variables no numéricas ("one hot encoding")

df = pd.get_dummies(df, columns = df.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)

pd.options.display.max_columns = None # remove the limit on the number of columns by default only 20 are shows

df.head()  # nuestro dataset tiene ahora 241 columnas (!)

In [ ]:
df.shape


# Paso 5:  Definición del vector objetivo (y) y la matriz de características (X)

In [ ]:
y = df['Retained.in.2012._1']
X = df.drop(columns = 'Retained.in.2012._1')

# Paso 6:  Dividir X, y en entrenamiento y prueba

In [ ]:
# Definimos la semilla para el generador de número aleatorios
np.random.seed(77300)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, stratify=y)
# IMPORTANTE: Las muestras están estratificadas, i.e., la proporción de clientes retenidos y no-retenidos es la misma en ambos

# Chequeemos los resultados
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)



In [ ]:
import seaborn as sns
sns.countplot(x=y_test, palette="bright")

# Pasos 7, 8, 9: Desarrollar un modelo con los datos de entrenamiento, Usarlo para predecir los valores en los datos de prueba, Calcular las métricas del modelo, y comparar modelos

In [ ]:
# Primero, definimos un conjunto de funciones para calcular las métricas del modelo

# Curva ROC
def plot_roc(y_test, y_pred):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1, drop_intermediate = False)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.001, 1.001])
    plt.ylim([-0.001, 1.001])
    plt.xlabel('1-Specificity (False Negative Rate)')
    plt.ylabel('Sensitivity (True Positive Rate)')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()

# Matriz de Confusión: cm[0,0], cm[0,1], cm[1,0], cm[1,1]: tn, fp, fn, tp

# Sensitivity
def custom_sensitivity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fn))

# Specificity
def custom_specificity_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fp))

# Positive Predictive Value
def custom_ppv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tp/(tp+fp))

# Negative Predictive Value
def custom_npv_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return (tn/(tn+fn))

# Accuracy
def custom_accuracy_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    return ((tn+tp)/(tn+tp+fn+fp))

# Modelo № 1: Support Vector Machines

In [ ]:
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

svm_estimators = []
svm_estimators.append(('normalize', MinMaxScaler())) # escalamos los datos
svm_estimators.append(('svm', svm.SVC(probability=True))) # definimos SVM con probabilidades 
     
# Definimos el modelo SVM y lo llamamos classifier_SVM
Classifier_SVM = Pipeline(svm_estimators, verbose=False)

# Entrenamos el modelo classifier_SVM sobre los datos de entrenamiento
Classifier_SVM.fit(X_train, y_train)

In [ ]:
# Usamos el modelo desarrollado, para predecir sobre los datos de prueba 
y_pred_prob = Classifier_SVM.predict_proba(X_test)[:,1] # probabilidades
class_threshold = 0.6073
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación
y_pred_prob

In [ ]:
# Usamos el modelo desarrollado, para predecir sobre los datos de prueba 
y_pred_prob = Classifier_SVM.predict_proba(X_test)[:,1] # probabilidades
class_threshold = 0.6073
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

# Revisemos las métricas del modelo

print('Métricas del modelo de Máquina de Vectores de Soporte: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de Confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))

In [ ]:
# Usamos el modelo desarrollado, para predecir sobre los datos de prueba 

y_pred_prob = Classifier_SVM.predict_proba(X_test)[:,1] # probabilidades
class_threshold = 0.5

y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

# Revisemos las métricas del modelo

print('Métricas del modelo de Máquina de Vectores de Soporte: \n')

cm = np.transpose(confusion_matrix(y_test, y_pred))
print("Matriz de Confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(y_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(y_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(y_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(y_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(y_test, y_pred)))

plot_roc(y_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(y_test, y_pred_prob)))

## Interpretación de la curva ROC y el AUC: 

- Si tomamos un cliente con una probabilidad pronósticada de 99%, uno con una probabilidad de 50%, y uno con una probabilidad del 1%. Esperaríamos que el modelo nos diera una certeza de que el primer cliente será retenido, y una certeza de que el tercero no lo será, pero podríamos estar cómodos con un error grande para el segundo.

- La curva ROC muestra justamente eso. Ranquea todos los clientes desde el que tiene la probabilidad más alta de retención hasta la más baja (equivale a variar el umbral desde alto hasta bajo). Comenzando en el origen, mapea todos los clientes en orden descendiente de probabilidad (desde el “mejor” hasta el “peor”). Un clasificador perfecto, con exactitud 100%, primero predeciría correctamente todos los positivos, y luego predeciría correctamente todos los negativos; es decir, la curva iría recto hasta el punto (0,1), y luego cambiaría y sería horizontal hasta el punto (1,1). Esto, por supuesto, no es posible en la práctica, y los “pasos” en la curva reflejan los errores ocasionales que el modelo comete. Un buen modelo cometería pocos errores positivos para los mejores clientes y pocos eroreres negativos para los peores.

-  Es importante observar que un modelo que simplemente adivina al azar, tendrá como curva ROC una línea de 45 grados. Tal modelo tendría la misma probabilidad de hacer una predicción correcta que una incorrecta, sin importar si el cliente tiene una alta o baja probabilidad predecida.

- En este caso el AUC es 86.26% el cual es un buen resultado. El AUC indica la proporción de parejas concordantes en los datos; en este caso el porcentaje de parejas concordantes es aproximadamente 86.26%. Las parejas concordantes son aquellas parejas de casos positivo y negativo en el dataset para las cuales el modelo de SVM - con ciertos parámetros (UMBRAL) - puede clasificarlos correctamente.

- En el dataset de prueba, tenemos 289 positivos (clientes retenidos) y 189 negativos (clientes no retenidos); el número total de parejas (positivos y negativos) es 289 x 189 = 54621, de los cuales el 86.26 % (== 47116) tienen unos parámetros (UMBRAL) del modelo de SVM que pueden clasificarlos correctamente.

In [ ]:
# Paso 3: Cargar los datos desde el archivo CSV en el dataframe llamado df_B.

df_B = pd.read_csv(r"C:\Users\carlo\Downloads\04 CSV data -- STC(B)_numerical dates.csv", sep=";", header = 0)  
df_B.head() # show the "head" -- first 5 rows of the data; note, these are rows 0...4

In [ ]:
df_C = pd.merge(df, df_B, on='ID',  how='left')
df_C.head()

In [ ]:
df_C.info

In [ ]:
# Método:
# Variables Categóricas: agregar una nueva categoría 'missing_value' (como si fuera un nuevo color, o género)
# Variables Numéricas: reemplazar con la mediana (o la media, o el valor más frecuente, etc.) Un método alterno es
# ejecutar una imputación, see here: https://scikit-learn.org/stable/modules/impute.html 
# + agregamos columnas sustitutas indicando que el valor ha sido imputado

# creación de variables sustitutas
for col in df_C:
    if df_C[col].isna().sum() != 0: 
        df_C[col + '_surrogate'] = df_C[col].isna().astype(int)

# fijación de variables categóricas
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(df_C.select_dtypes(exclude=['int64','float64']))
df_C[df_C.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(df_C.select_dtypes(exclude=['int64','float64']))
           
# fijación de variables numéricas 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(df_C.select_dtypes(include=['int64','float64']))
df_C[df_C.select_dtypes(include=['int64','float64']).columns] = imputer.transform(df_C.select_dtypes(include=['int64','float64']))

# Examinemos los resultados para la variable "Poverty.Code"
df_C[['NPS 2011','NPS 2011_surrogate']]

In [ ]:
yC = df_C['Retained.in.2012._1']
XC = df_C.drop(columns = 'Retained.in.2012._1')

In [ ]:
# Definimos la semilla para el generador de número aleatorios
np.random.seed(77300)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
XC_train, XC_test, yC_train, yC_test = train_test_split(XC, yC, test_size = 0.20, stratify=y)
# IMPORTANTE: Las muestras están estratificadas, i.e., la proporción de clientes retenidos y no-retenidos es la misma en ambos

# Chequeemos los resultados
print(XC_train.shape)
print(XC_test.shape)
print(yC_train.shape)
print(yC_test.shape)

In [ ]:
###  Ejecución de SVM con todas las variables del dataset completo, y evaluar desempeño

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
XCS_train = scaler.fit_transform(XC_train)
XCS_test = scaler.fit_transform(XC_test)

print(XCS_train)
print(XCS_test)

In [ ]:
# Desarrollamos un clasificador SVM LINEAL
clfl = svm.SVC(kernel='linear', C=1, probability=True, random_state=1234) # Linear Kernel

# Entrenamos el modelo usando el dataset de entrenamiento
clfl.fit(XCS_train, yC_train)

# Importamos el módulo scikit-learn metrics para cálculo de Accuracy
from sklearn import metrics

#Pronosticamos la respuesta para el dataset de prueba
#En entrenamiento
y_predl_e = clfl.predict(XCS_train)
#En prueba
y_predl = clfl.predict(XCS_test)

#Accuracy del modelo:  Qué tan frecuentemente el clasificador es correcto?
#En entrenamiento: 
print("Accuracy - entrenamiento: ",metrics.accuracy_score(yC_train, y_predl_e))
#En prueba: 
print("Accuracy - prueba: ",metrics.accuracy_score(yC_test, y_predl))


In [ ]:
from sklearn.metrics import confusion_matrix

# Revisemos las métricas del modelo en entrenamiento

print('Métricas del modelo de Máquina de Vectores de Soporte (ENTRENAMIENTO): \n')

cml_e = np.transpose(confusion_matrix(yC_train, y_predl_e))
print("Matriz de Confusión: \n" + str(cml_e))

print("                                   Accuracy: " + str(custom_accuracy_score(yC_train, y_predl_e))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(yC_train, y_predl_e)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(yC_train, y_predl_e)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(yC_train, y_predl_e)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(yC_train, y_predl_e)))

# Revisemos las métricas del modelo en prueba

print('\n \nMétricas del modelo de Máquina de Vectores de Soporte (PRUEBA): \n')

cml = np.transpose(confusion_matrix(yC_test, y_predl))
print("Matriz de Confusión: \n" + str(cml))

print("                                   Accuracy: " + str(custom_accuracy_score(yC_test, y_predl))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(yC_test, y_predl)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(yC_test, y_predl)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(yC_test, y_predl)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(yC_test, y_predl)))


y_pred_prob = clfl.predict_proba(XCS_test)[:,1] # probabilidades
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

plot_roc(yC_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(yC_test, y_pred_prob)))

In [ ]:
###  Ejecución de SVM con las 20 variables más importantes (usando RFE), y evaluar desempeño

In [ ]:
### REFERENCIAS SOBRE RFE (Recursive Feature Elimination)
### https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15
### https://www.kite.com/python/docs/sklearn.feature_selection.RFE

from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE

# create the RFE model for the svm classifier and select attributes
rfeSVM = RFE(estimator=clfl, n_features_to_select=20, step=1) 
rfeSVM.fit(XCS_train, yC_train)

ranking = rfeSVM.ranking_.reshape(len(XC_train.columns))

# Cuáles son las 20 variables que quedan en el modelo?
pd.DataFrame([XC_test.columns,ranking]).transpose().sort_values(1).head(30)

In [ ]:
# Entrenemos el nuevo modelo y llamemoslo classifier_SVML_RFE 
classifier_SVML_RFE = rfeSVM.fit(XCS_train, yC_train)

# Usemos el modelo entrenado para predecir sobre los datos de prueba
y_pred_prob = classifier_SVML_RFE.predict_proba(XCS_test)[:,1] # probabilidades
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

# Revisemos las métricas del modelo después de la selección de variables 
print('Métricas del modelo de SVM después de la selección de variables: \n')

cm = np.transpose(confusion_matrix(yC_test, y_pred))
print("Matriz de confusión: \n" + str(cm))

print("                                   Accuracy: " + str(custom_accuracy_score(yC_test, y_pred))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(yC_test, y_pred)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(yC_test, y_pred)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(yC_test, y_pred)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(yC_test, y_pred)))

plot_roc(yC_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(yC_test, y_pred_prob)))

In [ ]:
### Con el modelo de 20 variables, desarrollamos un SVM radial sintonizando hiper-parámetros

In [ ]:
XCS_train.shape

In [ ]:
XCS20_train = XCS_train[:, []]
XCS20_test = XCS_test[:, []]

In [ ]:
# Desarrollamos un clasificador SVM RADIAL
clfr = svm.SVC(kernel='rbf', C=1, probability=True, random_state=1234) 

# Entrenamos el modelo usando el dataset de entrenamiento
clfr.fit(XCS20_train, yC_train)

#Pronosticamos la respuesta para el dataset de prueba
#En entrenamiento
y_predr_e = clfr.predict(XCS20_train)
#En prueba
y_predr = clfr.predict(XCS20_test)

#Accuracy del modelo:  Qué tan frecuentemente el clasificador es correcto?
#En entrenamiento: 
print("Accuracy - entrenamiento: ",metrics.accuracy_score(yC_train, y_predr_e))
#En prueba: 
print("Accuracy - prueba: ",metrics.accuracy_score(yC_test, y_predr))

In [ ]:
from sklearn.metrics import confusion_matrix

# Revisemos las métricas del modelo en entrenamiento

print('Métricas del modelo de Máquina de Vectores de Soporte (ENTRENAMIENTO): \n')

cmr_e = np.transpose(confusion_matrix(yC_train, y_predr_e))
print("Matriz de Confusión: \n" + str(cmr_e))

print("                                   Accuracy: " + str(custom_accuracy_score(yC_train, y_predr_e))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(yC_train, y_predr_e)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(yC_train, y_predr_e)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(yC_train, y_predr_e)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(yC_train, y_predr_e)))

# Revisemos las métricas del modelo en prueba

print('\n \nMétricas del modelo de Máquina de Vectores de Soporte (PRUEBA): \n')

cmr = np.transpose(confusion_matrix(yC_test, y_predr))
print("Matriz de Confusión: \n" + str(cmr))

print("                                   Accuracy: " + str(custom_accuracy_score(yC_test, y_predr))) 
print("                       SENSITIVITY (RECALL): " + str(custom_sensitivity_score(yC_test, y_predr)))
print("                     SPECIFICITY (FALL-OUT): " + str(custom_specificity_score(yC_test, y_predr)))
print("     POSITIVE PREDICTIVE VALUE, (PRECISION): " + str(custom_ppv_score(yC_test, y_predr)))
print("                  NEGATIVE PREDICTIVE VALUE: " + str(custom_npv_score(yC_test, y_predr)))


y_pred_prob = clfr.predict_proba(XCS20_test)[:,1] # probabilidades
y_pred = np.where(y_pred_prob > class_threshold, 1, 0) # clasificación

plot_roc(yC_test, y_pred_prob)
print(" AUC: " + str(roc_auc_score(yC_test, y_pred_prob)))

## Resumen del SVM: 

- entrenamos el modelo con los hiper-parámetros por defecto
- aplicando el modelo al dataset de prueba, obtuvimos AUC=  %
- ahora, sintonizaremos los hiper-parámetros del modelo ...